# Project 3

## 1) Exploring data

In [94]:
import pandas as pd
import numpy as np
raw_data = pd.read_csv('ENB2012_data.csv')

In [95]:
raw_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [96]:
raw_data.shape

(768, 10)

In [97]:
raw_data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307201,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090196,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [98]:
raw_data.isnull().values.any()

False

In [99]:
x_t1 = raw_data.drop('Y1', axis=1)
x = x_t1.drop('Y2', axis=1)
y_1 = raw_data['Y1']
y_2 = raw_data['Y2']

## 2) Part 1: Running models for Heating and Cooling loads

### 2.1) Prepering data for Heating Load

In [100]:
from sklearn.model_selection import train_test_split

# default is 75% / 25% train-test split
x_train_orig, x_test_orig, y_train, y_test = train_test_split(x, y_1, random_state=0)

In [101]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train_orig)
x_test = scaler.transform(x_test_orig)

#### 2.1.1 KNN Regression

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
param_grid = {'n_neighbors':[1, 5, 7, 10, 15]}
grid_search = GridSearchCV(knn, param_grid)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'n_neighbors': 7}
0.9278282806768371


In [103]:
from sklearn.metrics import r2_score

knn = KNeighborsRegressor(n_neighbors=7).fit(x_train, y_train)
y_predict = knn.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

r2_score: 0.92461


#### 2.1.2 Linear Regression

In [104]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(x_train, y_train)
print("Training score: ",lr.score(x_train , y_train))
y_predict = lr.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

Training score:  0.9173512129865965
r2_score: 0.91129


#### 2.1.3 Gradient Boosting

In [105]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor().fit(x_train, y_train)
print('Training Score', gb.score(x_train, y_train))
y_predict = gb.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

Training Score 0.99844846657622
r2_score: 0.99736


#### 2.1.4 ADABoost

In [106]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(KNeighborsRegressor(n_neighbors=7), 
        n_estimators=200, learning_rate=0.5, random_state=42).fit(x_train, y_train)
print('Training Score', ada_reg.score(x_train, y_train))
y_predict = ada_reg.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))


Training Score 0.9640555509200671
r2_score: 0.95918


#### 2.1.5 Bagging with Decision Tree

In [107]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor().fit(x_train, y_train)
print('Training Score', bag_reg.score(x_train, y_train))
y_predict = bag_reg.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))



Training Score 0.9995881621221753
r2_score: 0.99688


#### 2.1.6 Deep Learning Resression

In [108]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(13, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mse', optimizer='adam' , metrics = ['mse'])
model.fit(x_train, y_train, epochs = 100, batch_size = 20)

y_train_predict = model.predict(x_train)
y_test_predict = model.predict(x_test)
print('Train r2: {:.2f}'.format(r2_score(y_train, y_train_predict)))
print('Test r2: {:.2f}'.format(r2_score(y_test, y_test_predict)))


Epoch 1/100
576/576 [==============================] - 0s 504us/step - loss: 589.0859 - mean_squared_error: 589.0859
Epoch 2/100
576/576 [==============================] - 0s 84us/step - loss: 585.8312 - mean_squared_error: 585.8312
Epoch 3/100
576/576 [==============================] - 0s 66us/step - loss: 577.5478 - mean_squared_error: 577.5478
Epoch 4/100
576/576 [==============================] - 0s 61us/step - loss: 559.3247 - mean_squared_error: 559.3247
Epoch 5/100
576/576 [==============================] - 0s 54us/step - loss: 524.4413 - mean_squared_error: 524.4413
Epoch 6/100
576/576 [==============================] - 0s 66us/step - loss: 469.9506 - mean_squared_error: 469.9506
Epoch 7/100
576/576 [==============================] - 0s 54us/step - loss: 394.0028 - mean_squared_error: 394.0028
Epoch 8/100
576/576 [==============================] - 0s 66us/step - loss: 303.7010 - mean_squared_error: 303.7010
Epoch 9/100
576/576 [==============================] - 0s 64us/step - l

### 2.2) Prepering data for Cooling Load

In [109]:
from sklearn.model_selection import train_test_split

# default is 75% / 25% train-test split
x_train_orig, x_test_orig, y_train, y_test = train_test_split(x, y_2, random_state=0)

In [110]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train_orig)
x_test = scaler.transform(x_test_orig)

#### 2.2.1 KNN Regression

In [111]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
param_grid = {'n_neighbors':[1, 5, 7, 10, 15]}
grid_search = GridSearchCV(knn, param_grid)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'n_neighbors': 7}
0.904699837697031


In [112]:
from sklearn.metrics import r2_score

knn = KNeighborsRegressor(n_neighbors=7).fit(x_train, y_train)
y_predict = knn.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

r2_score: 0.90959


#### 2.2.2 Linear Regression

In [113]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(x_train, y_train)
print("Training score: ",lr.score(x_train , y_train))
y_predict = lr.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

Training score:  0.8864209149755682
r2_score: 0.89100


#### 2.2.3 Gradient Boosting

In [114]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor().fit(x_train, y_train)
print('Training Score', gb.score(x_train, y_train))
y_predict = gb.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

Training Score 0.981647084317214
r2_score: 0.97542


#### 2.2.4 ADABoost

In [115]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(KNeighborsRegressor(n_neighbors=7), 
        n_estimators=200, learning_rate=0.5, random_state=42).fit(x_train, y_train)
print('Training Score', ada_reg.score(x_train, y_train))
y_predict = ada_reg.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))

Training Score 0.9508164546176986
r2_score: 0.92563


#### 2.2.5 Bagging with Decision Tree

In [116]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor().fit(x_train, y_train)
print('Training Score', bag_reg.score(x_train, y_train))
y_predict = bag_reg.predict(x_test)
print("r2_score: {:.5f}".format(r2_score(y_test, y_predict)))


Training Score 0.9941123486352204
r2_score: 0.96382


#### 2.2.6 Deep Learning Regression

In [117]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(13, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer = 'normal', activation = 'linear'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mse', optimizer='adam' , metrics = ['mse'])
model.fit(x_train, y_train, epochs = 100, batch_size = 20)

y_train_predict = model.predict(x_train)
y_test_predict = model.predict(x_test)
print('Train r2: {:.2f}'.format(r2_score(y_train, y_train_predict)))
print('Test r2: {:.2f}'.format(r2_score(y_test, y_test_predict)))


Epoch 1/100
576/576 [==============================] - 0s 542us/step - loss: 689.2033 - mean_squared_error: 689.2033
Epoch 2/100
576/576 [==============================] - 0s 68us/step - loss: 686.3490 - mean_squared_error: 686.3490
Epoch 3/100
576/576 [==============================] - 0s 66us/step - loss: 678.6508 - mean_squared_error: 678.6508
Epoch 4/100
576/576 [==============================] - 0s 68us/step - loss: 659.9304 - mean_squared_error: 659.9304
Epoch 5/100
576/576 [==============================] - 0s 52us/step - loss: 623.5931 - mean_squared_error: 623.5931
Epoch 6/100
576/576 [==============================] - 0s 68us/step - loss: 564.7773 - mean_squared_error: 564.7773
Epoch 7/100
576/576 [==============================] - 0s 68us/step - loss: 483.1012 - mean_squared_error: 483.1012
Epoch 8/100
576/576 [==============================] - 0s 70us/step - loss: 384.6476 - mean_squared_error: 384.6476
Epoch 9/100
576/576 [==============================] - 0s 52us/step - l

## 3) Part 2: Running classification model for High/Meduim/Low load efficiency

### 3.1 Creating dataset and required variables

In [118]:
raw_data_2 = raw_data

In [119]:
raw_data_2['Y'] = raw_data_2['Y1'] + raw_data_2['Y2']

In [120]:
raw_data_2.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,Y
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33,36.88
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33,36.88
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33,36.88
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33,36.88
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28,49.12


In [121]:
raw_data_2.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,Y
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307201,24.587760,46.894961
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090196,9.513306,19.484947
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000,16.950000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000,28.750000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000,40.970000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500,64.335000
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000,89.950000


In [122]:
def f(row):
    if row['Y'] > 64.335:
        val = 2
    elif row['Y'] < 28.75:
        val = 0
    else:
        val = 1
    return val

raw_data_2['Y_label'] = raw_data_2.apply(f, axis=1)
raw_data_2.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,Y,Y_label
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33,36.88,1
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33,36.88,1
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33,36.88,1
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33,36.88,1
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28,49.12,1


In [123]:
y_label = raw_data['Y_label']

In [124]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_label)
encoded_y = encoder.transform(y_label)
y_label_new = np_utils.to_categorical(encoded_y)
y_label_new

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [125]:
from sklearn.model_selection import train_test_split

x_train_orig, x_test_orig, y_train, y_test = train_test_split(x, y_label_new, random_state=0)

In [126]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train_orig)
x_test = scaler.transform(x_test_orig)

### 3.2 Creating, compiling and fitting the Deep Learning model

In [127]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu')) #hidden layer
model.add(Dense(3, activation='sigmoid')) #output layer

In [128]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [129]:
# Fit the model
model.fit(x_train, y_train, epochs=100, batch_size=20)

Epoch 1/100
576/576 [==============================] - 0s 593us/step - loss: 1.0494 - acc: 0.5104
Epoch 2/100
576/576 [==============================] - 0s 71us/step - loss: 1.0223 - acc: 0.5156
Epoch 3/100
576/576 [==============================] - 0s 73us/step - loss: 0.9953 - acc: 0.5208
Epoch 4/100
576/576 [==============================] - 0s 70us/step - loss: 0.9670 - acc: 0.5278
Epoch 5/100
576/576 [==============================] - 0s 64us/step - loss: 0.9383 - acc: 0.5330
Epoch 6/100
576/576 [==============================] - 0s 63us/step - loss: 0.9087 - acc: 0.5486
Epoch 7/100
576/576 [==============================] - 0s 78us/step - loss: 0.8787 - acc: 0.5503
Epoch 8/100
576/576 [==============================] - 0s 75us/step - loss: 0.8483 - acc: 0.5729
Epoch 9/100
576/576 [==============================] - 0s 71us/step - loss: 0.8186 - acc: 0.5833
Epoch 10/100
576/576 [==============================] - 0s 54us/step - loss: 0.7888 - acc: 0.6024
Epoch 11/100
576/576 [======

### 3.3 Model evaluation

In [130]:
# evaluate the model
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 640us/step

acc: 69.27%
